In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

%matplotlib inline

In [ ]:
#From Jose
gene = pd.read_excel('gene_expression_table4.xlsx')
del gene['Unnamed: 9']
# deleted column without any content
#append each column except the first column which is the gene ID
just_expression=gene.iloc[0:373,3:15]
just_expression


In [ ]:
JEflip = just_expression.transpose()
JEflip

JEflipar = JEflip.values
JEflipar

In [ ]:
#Making a new array to label gene expression under low or high BRCA1 (rows in JEflip)
brca = np.array(['BRCA1-','BRCA1-','BRCA1-','BRCA1-','BRCA1-','BRCA1-',
         'BRCA1+','BRCA1+','BRCA1+','BRCA1+','BRCA1+','BRCA1+'])

In [ ]:
from sklearn.decomposition import PCA

In [ ]:
JEflip_pca = PCA()
JEflippc = JEflip_pca.fit_transform(JEflipar) # 12x12 matrix

In [ ]:
fg = plt.figure()


fg.add_subplot(1,2,1)
plt.plot(np.cumsum(JEflip_pca.explained_variance_ratio_))
plt.xlabel("pc's")
plt.ylabel('fraction var explained')


JEvar = JEflip_pca.explained_variance_ratio_
cumvar = np.cumsum(JEvar) 
for i in range (12):
    if cumvar[i] >= 0.9:
        print (i+1, 'components are need to explain 90% of the variance')
        break
        
# 5 of the 12 PCs needed to explain 90% of the variance
# but the first PC already explains more than 75% of the variance

In [ ]:
# Plot function from homework 5 key

fg = plt.figure()

fg.add_subplot(3,1,1)
for c in np.unique(brca):
    plt.plot(JEflippc[brca==c,0], 
             JEflippc[brca==c,1], 'o')
plt.xlabel('PC1')    
plt.ylabel('PC2')

fg.add_subplot(3,1,2)
for c in np.unique(brca):
    plt.plot(JEflippc[brca==c,0], 
             JEflippc[brca==c,5], 'o')
plt.xlabel('PC1')    
plt.ylabel('PC6')

fg.add_subplot(3,1,3)
for c in np.unique(brca):
    plt.plot(JEflippc[brca==c,0], 
             JEflippc[brca==c,11], 'o')
plt.xlabel('PC1')    
plt.ylabel('PC12')

plt.tight_layout()

#When the indiviual PCs are compoared with the first PC, always a clear separation into 
#2 groups of 6 samples - corresponds to BRCA1+ or BRCA1-.
#Here example of 3

In [ ]:
#Function from homework 5 key

for c in np.unique(brca):
    plt.plot(JEflippc[brca==c,1], 
             JEflippc[brca==c,2], 'o')
plt.xlabel('PC2')    
plt.ylabel('PC3')

# When compared between other PCs that are not the first PC, less clear separation

In [ ]:
def test_train_id (N, test_frac):
    test = np.random.choice(np.arange(0, N), size = (np.floor(N * test_frac)), replace = False, p = None)
    train = np.setdiff1d(np.arange(0,N), test)
    return train, test

In [ ]:
# Function from homework 5 key 

from sklearn.decomposition import PCA

def cross_val_class_accuracy(model, X, y, r, test_frac, reps):
    JEflip_pca = PCA()
    
    score = np.array([])
    for i in range(reps):
        train, test = test_train_id(y.size, test_frac)
        y_train = y[train]
        y_test = y[test]
        
        X_train = JEflip_pca.fit_transform(X[train, :])[:, :r]
        X_test = JEflip_pca.transform(X[test, :])[:, :r]
        
        model.fit(X_train, y_train)
        
        pred = model.predict(X_test)
        this_score = sum(pred == y_test) / len(y_test)
        
        score = np.append(score, this_score)
    return score

In [ ]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.neighbors import KNeighborsClassifier
from sklearn import svm
from sklearn.tree import DecisionTreeClassifier

In [ ]:
score = cross_val_class_accuracy(LinearDiscriminantAnalysis(), JEflipar, 
                                brca, 10, 0.2, 200)
print(score.mean())

In [ ]:
testfrac = 1/6 
reps = 200

lda_r5_scores = cross_val_class_accuracy(LinearDiscriminantAnalysis(), 
                                          JEflipar, brca, 5,
                                          testfrac, reps)
lda_r10_scores = cross_val_class_accuracy(LinearDiscriminantAnalysis(), 
                                          JEflipar, brca, 10,
                                          testfrac, reps)
knn2_scores = cross_val_class_accuracy(KNeighborsClassifier(n_neighbors=2),
                                       JEflipar, brca, 10,
                                       testfrac, reps)
knn10_scores = cross_val_class_accuracy(KNeighborsClassifier(n_neighbors=10),
                                       JEflipar, brca, 10,
                                        testfrac, reps)
svm_scores = cross_val_class_accuracy(svm.SVC(kernel='linear'),
                                      JEflipar, brca, 10, 
                                      testfrac, reps)
tree_scores = cross_val_class_accuracy(DecisionTreeClassifier(max_depth=3),
                                       JEflipar, brca, 10,
                                       testfrac, reps)

In [ ]:
classifiers = ('lda_r5', 'lda_r10', 'knn2', 'knn10', 'svm', 'tree')
scores = [lda_r5_scores.mean(), lda_r10_scores.mean(), 
          knn2_scores.mean(), knn10_scores.mean(),
         svm_scores.mean(), tree_scores.mean()]
scores_err = [lda_r5_scores.std(), lda_r10_scores.std(), 
          knn2_scores.std(), knn10_scores.std(),
         svm_scores.std(), tree_scores.std()]


plt.bar(range(len(classifiers)), scores, 
        yerr=scores_err,
        align='center', alpha=0.4,)
plt.xticks(range(len(classifiers)), classifiers)
plt.ylabel('Cross-validated accuracy')
plt.xlabel('Classification Algorithm')

In [ ]:
brca2 = np.array([99,99,99,99,99,99,100,100,100,100,100,100])

brca3 = np.array ([1,1,1,1,1,1,0,0,0,0,0,0])

In [ ]:
from sklearn import linear_model
from sklearn.linear_model import Lasso
reg = linear_model.Lasso(alpha = 0.1)
reg.fit(JEflip, brca2)
print(reg.coef_)
output = reg.coef_

reg2 = linear_model.Lasso(alpha = 0.1)
reg2.fit(JEflip, brca3)
print(reg2.coef_)
output2 = reg2.coef_

output == output2

In [ ]:

clf = linear_model.Lasso(alpha=0.1)
clf.fit([[0,0], [1, 1], [2, 2]], [0, 1, 2])
Lasso(alpha=0.1, copy_X=True, fit_intercept=True, max_iter=1000,
   normalize=False, positive=False, precompute=False, random_state=None,
   selection='cyclic', tol=0.0001, warm_start=False)
print(clf.coef_)
print(clf.intercept_)



In [ ]:
train_ind1 = np.random.choice(np.arange(0,6), size=5, replace=False, p=None)
train_ind2 = np.random.choice(np.arange(6,12), size=5, replace=False, p=None)
print(train_ind1)
print(train_ind2)

In [ ]:
train_ind = np.concatenate((train_ind1, train_ind2), axis =0)
test_ind=  np.setdiff1d(np.arange(0,12), train_ind)

In [ ]:
train_ind

In [ ]:
test_ind

In [ ]:
expression_train=JEflipar[train_ind,:]
expression_test=JEflipar[test_ind,:]


In [ ]:
brca_end = np.array([0,0,0,0,0,0,1,1,1,1,1,1])

brcatrain = brca_end[train_ind]
brcatest = brca_end[test_ind]

#np.concatenate(expression_train,a)

In [ ]:
from sklearn.preprocessing import scale
from sklearn import cross_validation
from sklearn.linear_model import Ridge, RidgeCV, Lasso, LassoCV
from sklearn.metrics import mean_squared_error

lasso = Lasso(max_iter=10000, normalize=True)

In [ ]:
lassocv = LassoCV(alphas=None, cv=10, max_iter=100000, normalize=True)

lassocv.fit(expression_train, brcatrain)
lasso.set_params(alpha=lassocv.alpha_)
lasso.fit(expression_train, brcatrain)
mean_squared_error(brcatest, lasso.predict(expression_test))

In [ ]:
alphas = 10**np.linspace(10,-2,100)*0.5
lasso = Lasso(max_iter=10000, normalize=True)
coefs = []
for a in alphas:
    lasso.set_params(alpha=a)
    lasso.fit(scale(expression_train), brcatrain)
    coefs.append(lasso.coef_)
    
    #ax = plt.gca()
    #ax.plot(alphas*2, coefs)
    #ax.set_xscale('log')
    #plt.axis('tight')
    #plt.xlabel('tight')
    #plt.ylabel('weights')
    
#np.shape(coefs)
#v=coefs
#print(v)

ax = plt.gca()
ax.plot(alphas, coefs)
ax.set_xscale('log')
plt.axis('tight')
plt.xlabel('alpha')
plt.ylabel('weights')


print(lasso.coef_)


In [ ]:
lassocv = LassoCV(alphas=None, cv=10, max_iter=100000, normalize=True)

lassocv.fit(expression_train, brcatrain)
lasso.set_params(alpha=lassocv.alpha_)
lasso.fit(expression_train, brcatrain)
mean_squared_error(brcatest, lasso.predict(expression_test))

lasso.coef_